In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# faz a requisição
response = requests.get('https://pokemondb.net/pokedex/all')
if response.status_code == 200:
    print('Requisição bem-sucedida: 200')
else:
    print(f'Falha na requisição {response.status_code}')

# cria um objeto soup com o html
soup = BeautifulSoup(response.content)

# variaveis com a tags que contém as informações dos pokemons 
div = list(soup.body.main.find('div', {'class': 'grid-row'}))[1].find('div', {'class': 'resp-scroll'})
table = div.table

# faz uma lista com todas as tags <tr> (table row) da tag table
tr = []
for tag in table.find_all('tr'):
    if tag.name == 'tr':
        tr.append(tag)

# cada elemento  é uma tag tr cujo as filhas são tags td contém os dado dos pokemons (tr1 -> pokemon1, tr2 -> pokemon2, ...)
tr = tr[1:]

Requisição bem-sucedida: 200


In [2]:
# cria o dataframe
columns = ['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
df = pd.DataFrame(columns=columns)

# faz a separação dos dados 
keys = tuple(columns)
links = []
pokemons = []
composed_names = []

# itera sobre as tags tr (pokemons)
for i, poke in enumerate(tr):
    data = []
    # extrai os dados das tags td
    for j, info in enumerate(poke.children):
        # pula os elementos da lista que estão vazios
        if info != '\n' and info != ' ':
            if j == 3:
                # se for um nome composto e ele se repetir, ignora o primeiro
                if len(info.text.split(' ')) > 1:  
                    first = str(info.text.split(' ')[0])
                    rest = info.text.split(' ')[1:]
                    if first in rest:
                        name = ' '.join(rest)
                    else:
                        name = first + ' ' + ' '.join(rest)
                    data.append(name)
                    links.append({name: 'https://pokemondb.net' + info.a['href']})
                # se não for um nome composto, apenas faz append do nome
                else:
                    data.append(info.text)
                    links.append({info.text: 'https://pokemondb.net' + info.a['href']})
            # separa os tipos dos pokemons
            elif j == 4 and len(info.text.split(' ')) > 1:  
                type1, type2 = tuple(info.text.split(' '))
                if type2 == '':
                    type2 = None
                data.append(type1)
                data.append(type2)             
            # caso contrário, apenas faz append no dado
            else: 
                data.append(info.text)
 
    # cria um dicionário e coloca-o na lista
    pokemons.append(dict(zip(keys, data)))
    new_row = pd.DataFrame(dict(zip(keys, data)), index=[0])
    df = pd.concat([df, new_row], ignore_index=True)




In [30]:
for poke in pokemons:
    print(poke)

{'#': '0001', 'Name': 'Bulbasaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': '318', 'HP': '45', 'Attack': '49', 'Defense': '49', 'Sp. Atk': '65', 'Sp. Def': '65', 'Speed': '45'}
{'#': '0002', 'Name': 'Ivysaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': '405', 'HP': '60', 'Attack': '62', 'Defense': '63', 'Sp. Atk': '80', 'Sp. Def': '80', 'Speed': '60'}
{'#': '0003', 'Name': 'Venusaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': '525', 'HP': '80', 'Attack': '82', 'Defense': '83', 'Sp. Atk': '100', 'Sp. Def': '100', 'Speed': '80'}
{'#': '0003', 'Name': 'Mega Venusaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': '625', 'HP': '80', 'Attack': '100', 'Defense': '123', 'Sp. Atk': '122', 'Sp. Def': '120', 'Speed': '80'}
{'#': '0004', 'Name': 'Charmander', 'Type 1': 'Fire', 'Type 2': None, 'Total': '309', 'HP': '39', 'Attack': '52', 'Defense': '43', 'Sp. Atk': '60', 'Sp. Def': '50', 'Speed': '65'}
{'#': '0005', 'Name': 'Charmeleon', 'Type 1': 'Fire', 'Type 2': None, 'Total'

In [5]:
response = requests.get('https://pokemondb.net/pokedex/bulbasaur')
soup = BeautifulSoup(response.content)
response
        

<Response [200]>

In [3]:
links

[{'Bulbasaur': 'https://pokemondb.net/pokedex/bulbasaur'},
 {'Ivysaur': 'https://pokemondb.net/pokedex/ivysaur'},
 {'Venusaur': 'https://pokemondb.net/pokedex/venusaur'},
 {'Mega Venusaur': 'https://pokemondb.net/pokedex/venusaur'},
 {'Charmander': 'https://pokemondb.net/pokedex/charmander'},
 {'Charmeleon': 'https://pokemondb.net/pokedex/charmeleon'},
 {'Charizard': 'https://pokemondb.net/pokedex/charizard'},
 {'Mega Charizard X': 'https://pokemondb.net/pokedex/charizard'},
 {'Mega Charizard Y': 'https://pokemondb.net/pokedex/charizard'},
 {'Squirtle': 'https://pokemondb.net/pokedex/squirtle'},
 {'Wartortle': 'https://pokemondb.net/pokedex/wartortle'},
 {'Blastoise': 'https://pokemondb.net/pokedex/blastoise'},
 {'Mega Blastoise': 'https://pokemondb.net/pokedex/blastoise'},
 {'Caterpie': 'https://pokemondb.net/pokedex/caterpie'},
 {'Metapod': 'https://pokemondb.net/pokedex/metapod'},
 {'Butterfree': 'https://pokemondb.net/pokedex/butterfree'},
 {'Weedle': 'https://pokemondb.net/pokedex

In [4]:
soup.body.main.p.abbr.text

AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
for i in range(5):
    response = requests.get(links[i])
    soup = BeautifulSoup(response.content)
    if i == 2:
    
        display(soup.body.main.find_all('p')[1])

    #print(soup.body.main.p.em.text, soup.body.main.p.abbr.text)

IndexError: list index out of range